# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: dda01b26-dea3-477d-8e94-24b339ebb7e9
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session dda01b26-dea3-477d-8e94-24b339ebb7e9 to get into ready status...
Session dda01b26-dea3-477d-8e94-24b339ebb7e9 ha

Creating Glue Data Catalog Tables

In [2]:
import boto3
from datetime import datetime

In [3]:
# Initialize a session using AWS Glue
glue_client = boto3.client('glue', region_name='us-east-1')


In [4]:
# Step 1: Create a Glue Database (if it doesn't exist)
database_name = "bikeshare_data"
try:
    glue_client.create_database(DatabaseInput={'Name': database_name})
    print(f"Database '{database_name}' created.")
except glue_client.exceptions.AlreadyExistsException:
    print(f"Database '{database_name}' already exists.")

{'ResponseMetadata': {'RequestId': '2e2ecea8-ef0e-4bca-9393-9d7b870e35a7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 02 Dec 2024 05:30:59 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '2e2ecea8-ef0e-4bca-9393-9d7b870e35a7', 'cache-control': 'no-cache'}, 'RetryAttempts': 0}}
Database 'bikeshare_data' created.


In [5]:
# Step 2: Create Glue Table for capital_rideshare_combined.csv
table_name_1 = "capital_rideshare_combined"
columns_1 = [
    {"Name": "ride_id", "Type": "string"},
    {"Name": "rideable_type", "Type": "string"},
    {"Name": "started_at", "Type": "timestamp"},
    {"Name": "ended_at", "Type": "timestamp"},
    {"Name": "start_station_name", "Type": "string"},
    {"Name": "start_station_id", "Type": "int"},
    {"Name": "end_station_name", "Type": "string"},
    {"Name": "end_station_id", "Type": "int"},
    {"Name": "start_lat", "Type": "double"},
    {"Name": "start_lng", "Type": "double"},
    {"Name": "end_lat", "Type": "double"},
    {"Name": "end_lng", "Type": "double"},
    {"Name": "member_casual", "Type": "string"},
    {"Name": "year", "Type": "int"},
    {"Name": "month", "Type": "int"}
]


In [6]:
# Define the S3 location where the file is stored
s3_path_1 = "s3://capitalbikesharedata/capital_rideshare_combined.csv"

In [7]:
# Create Glue table for capital_rideshare_combined
glue_client.create_table(
    DatabaseName=database_name,
    TableInput={
        'Name': table_name_1,
        'StorageDescriptor': {
            'Columns': columns_1,
            'Location': s3_path_1,
            'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
            'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
            'SerdeInfo': {
                'Name': 'CSVSerde',
                'SerializationLibrary': 'org.apache.hadoop.hive.serde2.OpenCSVSerde',
                'Parameters': {
                    'separatorChar': ',',
                    'quoteChar': '"'
                }
            },
            'Compressed': False,
            'NumberOfBuckets': -1
        },
        'PartitionKeys': [],
        'TableType': 'EXTERNAL_TABLE',
    }
)

print(f"Table '{table_name_1}' created in the Glue Catalog.")


Table 'capital_rideshare_combined' created in the Glue Catalog.


In [8]:
# Step 3: Create Glue Table for placemarks_data.csv
table_name_2 = "placemarks_data"
columns_2 = [
    {"Name": "id", "Type": "string"},
    {"Name": "networkName", "Type": "string"},
    {"Name": "networkId", "Type": "int"},
    {"Name": "color", "Type": "string"},
    {"Name": "textColor", "Type": "string"},
    {"Name": "latitude", "Type": "double"},
    {"Name": "longitude", "Type": "double"},
    {"Name": "type", "Type": "string"},
    {"Name": "location", "Type": "string"}
]

In [9]:
# Define the S3 location where the file is stored
s3_path_2 = "s3://capitalbikesharedata/placemarks_data.csv"

In [10]:
# Create Glue table for placemarks_data
glue_client.create_table(
    DatabaseName=database_name,
    TableInput={
        'Name': table_name_2,
        'StorageDescriptor': {
            'Columns': columns_2,
            'Location': s3_path_2,
            'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
            'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
            'SerdeInfo': {
                'Name': 'CSVSerde',
                'SerializationLibrary': 'org.apache.hadoop.hive.serde2.OpenCSVSerde',
                'Parameters': {
                    'separatorChar': ',',
                    'quoteChar': '"'
                }
            },
            'Compressed': False,
            'NumberOfBuckets': -1
        },
        'PartitionKeys': [],
        'TableType': 'EXTERNAL_TABLE',
    }
)

print(f"Table '{table_name_2}' created in the Glue Catalog.")

Table 'placemarks_data' created in the Glue Catalog.
